In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

## Load Data

In [2]:
train = pd.read_csv("train.csv")
macro = pd.read_csv("macro.csv")

In [3]:
train.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [4]:
macro.head()

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,2010-01-01,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
1,2010-01-02,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
2,2010-01-03,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
3,2010-01-04,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.905,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
4,2010-01-05,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.836,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN


## Train Data cleaning

Here are the non-numeric data types for train. We want to remove sub_area because our training dataset cannot contain any dummified variables for the geographic location of the properties. 'id' and 'timestamp' can be removed because these won't contribute to the model. 

In [5]:
train.select_dtypes('O').head(2)

,timestamp,product_type,sub_area,culture_objects_top_25,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,water_1line,big_road1_1line,railroad_1line,ecology
0,2011-08-20,Investment,Bibirevo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2011-08-23,Investment,Nagatinskij Zaton,yes,no,no,no,no,no,no,no,no,no,no,no,excellent


In [6]:
train = train.drop(["id", "sub_area"], axis=1)

Now we want to fill in missing data, because there are a lot of observations with NA. I will just fill them in with the average of each column. 

In [7]:
train = train.fillna(train.mean())

Now we will fix some data quality issues.

The "state" column has a data entry error. Values should be discrete from 1-4, but there is one value with 33. I will replace the value with the mode. 

In [8]:
# round all values to integers
train.loc[:, "state"] = np.round(train["state"]).astype('int')

In [9]:
train.at[(train["state"] == 33).argmax(), "state"] = train["state"].mode()

The 'state' variable is coded as numerical but is actually categorical. Trick pandas into saving it as a string.

In [10]:
train['state'] = train['state'].apply(lambda n: "'" + str(n) + "'")

### Macro Data Cleaning

Here are the non-numeric columns for macro. 

In [11]:
macro.select_dtypes('O').head(2)

,timestamp,child_on_acc_pre_school,modern_education_share,old_education_build_share
0,2010-01-01,"45,713",NaN,NaN
1,2010-01-02,"45,713",NaN,NaN


The 3 columns above have weird values in them, seems like they're supposed to be numeric but have commas. Tried to look online but didn't find much information about what the columns actually mean. Will drop these from the dataset because they likely do not contribute much to sale price.

In [12]:
macro = macro.drop(["child_on_acc_pre_school", "modern_education_share", "old_education_build_share"], axis=1)

Now we want to fill in missing data, because there are a lot of observations with NA. I will just fill them in with the average of each column. 

In [13]:
macro = macro.fillna(macro.mean())

## Combine data into one dataframe on timestamp

In [14]:
train_columns = set(train.columns)
macro_columns = set(macro.columns)

In [15]:
train["timestamp"] = train["timestamp"].astype("string")

In [16]:
macro["timestamp"] = macro["timestamp"].astype('string')

In [17]:
train_macro = train.join(macro.set_index('timestamp'), on="timestamp", how="left", rsuffix="_macro")

# drop timestamp because it is no longer needed
train_macro = train_macro.drop("timestamp", axis=1)

In [18]:
# check for NA in train_macro
train_macro.isna().sum().sum()

0

## Standardizing Data

Standardize train + macro data

In [19]:
# only standardize the numeric columns
train_macro_numeric_columns = train_macro.select_dtypes(['int', 'float']).columns

In [20]:
standard_scaler = preprocessing.StandardScaler()
train_macro[train_macro_numeric_columns] = standard_scaler.fit_transform(train_macro[train_macro_numeric_columns])

## Save data for analysis in R

In [21]:
# after standardizing above, separate train and macro into separate files again for analysis outside of this notebook 
train_subset = train_macro.filter(train.columns, axis=1)
macro_subset = train_macro.filter(macro.columns, axis=1)

In [22]:
train_subset.to_csv("/train_cleaned.csv", index=False)
macro_subset.to_csv("/macro_cleaned.csv", index=False)
train_macro.to_csv("/train_macro.csv", index=False)

# Selecting the best features using only data from Train

In [2]:
train = pd.read_csv("train_cleaned.csv")
macro = pd.read_csv("macro_cleaned.csv")
train_macro = pd.read_csv("train_macro.csv")

In [3]:
X_train = pd.get_dummies(train.drop("price_doc", axis=1)) 
y_train = train["price_doc"]

max_train_features = 40
max_macro_features = 10
max_total_features = max_train_features + max_macro_features

## Univariate Feature Selection

In [4]:
fit = SelectKBest(f_regression, k=max_train_features).fit(X_train, y_train)
feature_scores = pd.Series(fit.scores_, index=X_train.columns)
top_40_features_k_best = feature_scores.nlargest(n=max_train_features).index

In [5]:
top_40_features_k_best

Index(['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
       'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'kremlin_km',
       'bulvar_ring_km', 'sport_count_2000', 'ttk_km', 'office_sqm_5000',
       'trc_sqm_5000', 'sport_count_1500', 'nuclear_reactor_km',
       'sport_objects_raion', 'trc_count_3000', 'cafe_count_5000_price_1000',
       'stadium_km', 'cafe_count_5000_price_1500', 'cafe_count_5000',
       'cafe_count_5000_na_price', 'ecology_no data',
       'cafe_count_5000_price_500', 'office_sqm_3000',
       'cafe_count_5000_price_2500', 'trc_sqm_3000', 'basketball_km',
       'office_km', 'detention_facility_km', 'office_count_5000',
       'university_km', 'office_sqm_2000', 'theater_km',
       'cafe_count_5000_price_high', 'church_count_5000', 'swim_pool_km',
       'catering_km', 'thermal_power_plant_km', 'cafe_count_5000_price_4000'],
      dtype='object')

## Feature Importance

In [6]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)

ExtraTreesRegressor(n_jobs=-1)

In [7]:
feature_scores = pd.Series(model.feature_importances_, index=X_train.columns)
top_40_features_feature_importance = feature_scores.nlargest(max_train_features).index

In [8]:
top_40_features_feature_importance

Index(['full_sq', 'num_room', 'life_sq', 'floor', 'state_'4'', 'kitch_sq',
       'cafe_count_3000_price_2500', 'sport_count_5000',
       'cafe_count_5000_price_2500', 'cafe_count_3000',
       'product_type_OwnerOccupier', 'state_'2'', 'max_floor',
       'cafe_count_2000_price_2500', 'cafe_count_5000_price_1500',
       'cafe_count_2000_price_1000', 'product_type_Investment',
       'cafe_count_5000', 'office_sqm_5000', 'cafe_count_5000_price_high',
       'cafe_count_3000_price_1500', 'trc_count_3000', 'material',
       'cafe_count_2000', 'build_year', 'kindergarten_km',
       'cafe_count_3000_price_1000', 'cafe_count_2000_price_1500',
       'sport_count_3000', 'cafe_count_1500', 'cafe_count_5000_price_1000',
       'trc_count_2000', 'industrial_km', 'state_'3'',
       'ID_railroad_station_walk', 'cafe_count_3000_price_high',
       'cafe_count_3000_price_500', 'cafe_count_1500_price_1000',
       'cafe_count_1000_price_2500', 'culture_objects_top_25_no'],
      dtype='object')

In [9]:
set(top_40_features_feature_importance).intersection(set(top_40_features_k_best))

{'cafe_count_5000',
 'cafe_count_5000_price_1000',
 'cafe_count_5000_price_1500',
 'cafe_count_5000_price_2500',
 'cafe_count_5000_price_high',
 'full_sq',
 'num_room',
 'office_sqm_5000',
 'sport_count_3000',
 'sport_count_5000',
 'trc_count_3000'}

## Correlation

In [10]:
top_40_features_correlation = train.corr()['price_doc'].nlargest(n=max_train_features+1).index[1:] # price_doc is included in correlation, so need top 41 features

In [11]:
top_40_features_correlation 

Index(['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
       'trc_count_5000', 'sport_count_2000', 'office_sqm_5000', 'trc_sqm_5000',
       'sport_count_1500', 'sport_objects_raion', 'trc_count_3000',
       'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500',
       'cafe_count_5000', 'cafe_count_5000_na_price',
       'cafe_count_5000_price_500', 'office_sqm_3000',
       'cafe_count_5000_price_2500', 'trc_sqm_3000', 'office_count_5000',
       'office_sqm_2000', 'cafe_count_5000_price_high', 'church_count_5000',
       'cafe_count_5000_price_4000', 'leisure_count_5000',
       'big_church_count_5000', 'sport_count_1000', 'office_sqm_1500',
       'market_count_5000', 'school_education_centers_raion',
       'healthcare_centers_raion', 'office_count_500', 'trc_count_2000',
       'ekder_male', 'cafe_count_500_price_1000', 'office_count_1000',
       'mosque_count_5000', 'ekder_all', 'cafe_count_1000_price_high',
       'ekder_female'],
      dtype='object')

# Selecting the best features using only data from Macro

In [12]:
X_train = pd.get_dummies(macro) # macro shouldn't have any categorical features
y_train = train["price_doc"]

## Univariate Feature Selection

In [13]:
fit = SelectKBest(f_regression, k=max_macro_features).fit(X_train, y_train)
feature_scores = pd.Series(fit.scores_, index=X_train.columns)
top_10_features_k_best = feature_scores.nlargest(n=max_macro_features).index

In [14]:
top_10_features_k_best

Index(['bandwidth_sports', 'fixed_basket', 'provision_doctors', 'cpi',
       'gdp_annual_growth', 'salary', 'gdp_deflator',
       'load_of_teachers_school_per_teacher', 'turnover_catering_per_cap',
       'deposits_value'],
      dtype='object')

## Feature Importance

In [15]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)

ExtraTreesRegressor(n_jobs=-1)

In [16]:
feature_scores = pd.Series(model.feature_importances_, index=X_train.columns)
top_10_features_feature_importance = feature_scores.nlargest(max_macro_features).index

In [17]:
top_10_features_feature_importance

Index(['micex_cbi_tr', 'eurrub', 'micex', 'brent', 'micex_rgbi_tr', 'usdrub',
       'rts', 'provision_doctors', 'employment', 'gdp_annual_growth'],
      dtype='object')

In [18]:
# does kbest and feature importance agree on the same variables?
set(top_10_features_feature_importance).intersection(set(top_10_features_k_best))

{'gdp_annual_growth', 'provision_doctors'}

## Correlation

In [19]:
macro = pd.concat([macro, y_train], axis=1) # add price_doc column to macro data 

In [20]:
top_10_features_correlation = macro.corr()['price_doc'].nlargest(n=max_macro_features+1).index[1:] # price_doc is included in correlation, so need top 41 features

In [21]:
top_10_features_correlation 

Index(['bandwidth_sports', 'fixed_basket', 'cpi', 'salary', 'gdp_deflator',
       'load_of_teachers_school_per_teacher', 'turnover_catering_per_cap',
       'deposits_value', 'gdp_annual', 'labor_force'],
      dtype='object')

# Selecting the best features using data from Train and Macro at the same time
We will select the best 50 features with no specific number of features from either dataset

In [22]:
X_train = pd.get_dummies(train_macro.drop("price_doc", axis=1)) 
y_train = train_macro["price_doc"]

In [23]:
def get_features(train_columns, macro_columns, all_columns):
    num_train_features = 40
    num_macro_features = 10
    
    train_features = []
    macro_features = []
       
    for i,col in enumerate(all_columns):
        if col in train_columns:
            #print(i,"train",col)
            train_features.append(col)
            
        elif col in macro_columns:
            #print(i,"macro",col)
            macro_features.append(col)
            
        if len(train_features) + len(macro_features) >= num_train_features + num_macro_features:
            break
            
    return train_features, macro_features

## Univariate Feature Selection

In [24]:
fit = SelectKBest(f_regression, k=max_total_features).fit(X_train, y_train)
feature_scores = pd.Series(fit.scores_, index=X_train.columns)
top_features_k_best = feature_scores.nlargest(n=max_total_features).index

In [25]:
top_features_k_best

Index(['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
       'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'kremlin_km',
       'bulvar_ring_km', 'sport_count_2000', 'ttk_km', 'office_sqm_5000',
       'trc_sqm_5000', 'sport_count_1500', 'nuclear_reactor_km',
       'sport_objects_raion', 'trc_count_3000', 'cafe_count_5000_price_1000',
       'stadium_km', 'cafe_count_5000_price_1500', 'cafe_count_5000',
       'cafe_count_5000_na_price', 'ecology_no data',
       'cafe_count_5000_price_500', 'office_sqm_3000',
       'cafe_count_5000_price_2500', 'trc_sqm_3000', 'basketball_km',
       'office_km', 'detention_facility_km', 'office_count_5000',
       'university_km', 'office_sqm_2000', 'theater_km',
       'cafe_count_5000_price_high', 'church_count_5000', 'swim_pool_km',
       'catering_km', 'thermal_power_plant_km', 'cafe_count_5000_price_4000',
       'workplaces_km', 'exhibition_km', 'leisure_count_5000',
       'big_church_count_5000', 'sport_count_1000',

In [26]:
train_features, macro_features = get_features(train.columns, macro.columns, top_features_k_best)

In [27]:
np.array(train_features)

array(['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
       'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'kremlin_km',
       'bulvar_ring_km', 'sport_count_2000', 'ttk_km', 'office_sqm_5000',
       'trc_sqm_5000', 'sport_count_1500', 'nuclear_reactor_km',
       'sport_objects_raion', 'trc_count_3000',
       'cafe_count_5000_price_1000', 'stadium_km',
       'cafe_count_5000_price_1500', 'cafe_count_5000',
       'cafe_count_5000_na_price', 'cafe_count_5000_price_500',
       'office_sqm_3000', 'cafe_count_5000_price_2500', 'trc_sqm_3000',
       'basketball_km', 'office_km', 'detention_facility_km',
       'office_count_5000', 'university_km', 'office_sqm_2000',
       'theater_km', 'cafe_count_5000_price_high', 'church_count_5000',
       'swim_pool_km', 'catering_km', 'thermal_power_plant_km',
       'cafe_count_5000_price_4000', 'workplaces_km', 'exhibition_km',
       'leisure_count_5000', 'big_church_count_5000', 'sport_count_1000',
       'museum_km'

In [28]:
np.array(macro_features)

array([], dtype=float64)

## Feature Importance

In [30]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)

ExtraTreesRegressor(n_jobs=-1)

In [31]:
feature_scores = pd.Series(model.feature_importances_, index=X_train.columns)
top_features_feature_importance = feature_scores.nlargest(max_total_features).index

In [32]:
top_features_feature_importance

Index(['full_sq', 'num_room', 'life_sq', 'state_'4'', 'kitch_sq',
       'cafe_count_3000', 'cafe_count_2000', 'cafe_count_5000_price_high',
       'sport_count_3000', 'cafe_count_3000_price_2500',
       'cafe_count_5000_price_2500', 'product_type_OwnerOccupier',
       'cafe_count_5000', 'max_floor', 'cafe_count_2000_price_2500',
       'cafe_count_5000_price_1500', 'cafe_count_3000_price_1000',
       'product_type_Investment', 'floor', 'office_sqm_5000',
       'cafe_count_3000_price_1500', 'state_'2'', 'cafe_count_5000_price_1000',
       'ID_railroad_station_walk', 'kindergarten_km', 'nuclear_reactor_km',
       'cafe_count_5000_price_500', 'trc_count_3000',
       'cafe_count_3000_price_high', 'cafe_count_2000_price_1000',
       'sport_count_5000', 'culture_objects_top_25_yes', 'trc_count_2000',
       'cafe_count_3000_price_500', 'office_count_5000', 'industrial_km',
       'culture_objects_top_25_no', 'cafe_count_1000_price_2500',
       'trc_sqm_3000', 'material', 'cafe_coun

In [33]:
# does kbest and feature importance agree on the same variables?
set(top_features_feature_importance[:40]).intersection(set(top_features_k_best[:40]))

{'cafe_count_5000',
 'cafe_count_5000_price_1000',
 'cafe_count_5000_price_1500',
 'cafe_count_5000_price_2500',
 'cafe_count_5000_price_500',
 'cafe_count_5000_price_high',
 'full_sq',
 'nuclear_reactor_km',
 'num_room',
 'office_count_5000',
 'office_sqm_5000',
 'sport_count_3000',
 'sport_count_5000',
 'trc_count_3000',
 'trc_sqm_3000'}

In [34]:
train_features, macro_features = get_features(train.columns, macro.columns, top_features_feature_importance)

In [35]:
np.array(train_features)

array(['full_sq', 'num_room', 'life_sq', 'kitch_sq', 'cafe_count_3000',
       'cafe_count_2000', 'cafe_count_5000_price_high',
       'sport_count_3000', 'cafe_count_3000_price_2500',
       'cafe_count_5000_price_2500', 'cafe_count_5000', 'max_floor',
       'cafe_count_2000_price_2500', 'cafe_count_5000_price_1500',
       'cafe_count_3000_price_1000', 'floor', 'office_sqm_5000',
       'cafe_count_3000_price_1500', 'cafe_count_5000_price_1000',
       'ID_railroad_station_walk', 'kindergarten_km',
       'nuclear_reactor_km', 'cafe_count_5000_price_500',
       'trc_count_3000', 'cafe_count_3000_price_high',
       'cafe_count_2000_price_1000', 'sport_count_5000', 'trc_count_2000',
       'cafe_count_3000_price_500', 'office_count_5000', 'industrial_km',
       'cafe_count_1000_price_2500', 'trc_sqm_3000', 'material',
       'cafe_count_5000_price_4000', 'sport_count_2000', 'trc_sqm_500',
       'trc_count_5000', 'cafe_count_2000_price_1500',
       'cafe_count_1500_price_2500', 'g

In [36]:
np.array(macro_features)

array(['rent_price_3room_eco'], dtype='<U20')

## Correlation

In [37]:
top_features_correlation = train_macro.corr()['price_doc'].nlargest(n=max_total_features+1).index[1:] # price_doc is included in correlation, so need top 41 features

In [38]:
top_features_correlation

Index(['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
       'trc_count_5000', 'sport_count_2000', 'office_sqm_5000', 'trc_sqm_5000',
       'sport_count_1500', 'sport_objects_raion', 'trc_count_3000',
       'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500',
       'cafe_count_5000', 'cafe_count_5000_na_price',
       'cafe_count_5000_price_500', 'office_sqm_3000',
       'cafe_count_5000_price_2500', 'trc_sqm_3000', 'office_count_5000',
       'office_sqm_2000', 'cafe_count_5000_price_high', 'church_count_5000',
       'cafe_count_5000_price_4000', 'leisure_count_5000',
       'big_church_count_5000', 'sport_count_1000', 'office_sqm_1500',
       'market_count_5000', 'school_education_centers_raion',
       'healthcare_centers_raion', 'office_count_500', 'trc_count_2000',
       'ekder_male', 'cafe_count_500_price_1000', 'office_count_1000',
       'mosque_count_5000', 'ekder_all', 'cafe_count_1000_price_high',
       'ekder_female', 'cafe_count_3000_price_10

In [39]:
train_features, macro_features = get_features(train.columns, macro.columns, top_features_correlation)

In [40]:
np.array(train_features)

array(['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
       'trc_count_5000', 'sport_count_2000', 'office_sqm_5000',
       'trc_sqm_5000', 'sport_count_1500', 'sport_objects_raion',
       'trc_count_3000', 'cafe_count_5000_price_1000',
       'cafe_count_5000_price_1500', 'cafe_count_5000',
       'cafe_count_5000_na_price', 'cafe_count_5000_price_500',
       'office_sqm_3000', 'cafe_count_5000_price_2500', 'trc_sqm_3000',
       'office_count_5000', 'office_sqm_2000',
       'cafe_count_5000_price_high', 'church_count_5000',
       'cafe_count_5000_price_4000', 'leisure_count_5000',
       'big_church_count_5000', 'sport_count_1000', 'office_sqm_1500',
       'market_count_5000', 'school_education_centers_raion',
       'healthcare_centers_raion', 'office_count_500', 'trc_count_2000',
       'ekder_male', 'cafe_count_500_price_1000', 'office_count_1000',
       'mosque_count_5000', 'ekder_all', 'cafe_count_1000_price_high',
       'ekder_female', 'cafe_count_3000_p

In [41]:
np.array(macro_features)

array([], dtype=float64)